# Case

In [ ]:
#case_pdac: case person ids with pdac concepts 
# SELECT co.person_id, co.condition_concept_id, co.condition_start_datetime
# FROM clinical_merge_v5_2022q1.condition_occurrence co
# WHERE co.condition_concept_id IN (
#     SELECT ca.descendant_concept_id
#     FROM clinical_merge_v5_2022q1.concept c
#     JOIN clinical_merge_v5_2022q1.concept_ancestor ca
#     ON ca.ancestor_concept_id = c.concept_id
#     WHERE c.concept_id IN (44830971, 1567481, 4180793))

#case: case all conditions
# SELECT co.person_id, co.condition_concept_id, co.condition_start_datetime
# FROM clinical_merge_v5_2022q1.condition_occurrence co
# WHERE co.person_id IN (
#     SELECT DISTINCT co.person_id
#     FROM clinical_merge_v5_2022q1.condition_occurrence co
#     JOIN clinical_merge_v5_2022q1.concept_ancestor ca
#     ON ca.descendant_concept_id = co.condition_concept_id
#     WHERE ca.ancestor_concept_id IN (44830971, 1567481, 4180793))

# case demog
# SELECT p.person_id, p.gender_concept_id,p.ethnicity_concept_id, p.race_concept_id, p.year_of_birth, p.month_of_birth, p.day_of_birth
# FROM clinical_merge_v5_2022q1.person p
# WHERE p.person_id IN (
#     SELECT DISTINCT co.person_id
#     FROM clinical_merge_v5_2022q1.condition_occurrence co
#     JOIN clinical_merge_v5_2022q1.concept_ancestor ca
#     ON ca.descendant_concept_id = co.condition_concept_id
#     WHERE ca.ancestor_concept_id IN (44830971, 1567481, 4180793))

In [5]:
import pandas as pd
case = pd.read_csv('E:/2023 EHR data/case_condition04082024.csv')

In [6]:
case_pdac = pd.read_csv('E:/2023 EHR data/case_pdac_condition04082024.csv')

In [7]:
print(len(case['person_id'].unique()))
print(len(case['condition_concept_id'].unique()))

12612
8796


### Remove concept id 0 - unknown

In [9]:
len(case)

2995875

In [10]:
case = case[case['condition_concept_id']!=0]
len(case)

2981227

In [11]:
print(len(case['person_id'].unique()))

12612


### pancreatic cancer first diagnosis date

In [12]:
from tqdm import tqdm
pc_diag = {}
patlst = list(case_pdac['person_id'].unique())
case_pdac['condition_start_datetime'] = pd.to_datetime(case_pdac['condition_start_datetime'])
grk = case_pdac.groupby('person_id')

for p in tqdm(patlst):
    dat = grk.get_group(p)
    dat = dat.sort_values(by='condition_start_datetime')
    pc_diag[p] = list(dat['condition_start_datetime'])[0]

100%|██████████| 12612/12612 [00:07<00:00, 1646.36it/s]


In [ ]:
import pickle
with open('datasets/pc_diag.pickle', 'wb') as handle:
    pickle.dump(pc_diag, handle)

### patients with minumum 5 data points before PC diagnosis

In [13]:
grk = case.groupby('person_id')
case['condition_start_datetime'] = pd.to_datetime(case['condition_start_datetime'], errors = 'coerce')

pre = []
for p in tqdm(patlst):
    dat = grk.get_group(p)
    dat = dat[~dat.duplicated()]
    dat = dat[dat['condition_start_datetime']<pc_diag[p]]
    if len(dat)>=5:
        pre.append(dat)

C:\Users\jp4147\AppData\Local\Temp\ipykernel_49128\460566437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case['condition_start_datetime'] = pd.to_datetime(case['condition_start_datetime'], errors = 'coerce')
100%|██████████| 12612/12612 [00:18<00:00, 678.00it/s]


In [14]:
pre = pd.concat(pre)
len(pre['person_id'].unique())

6990

### case demographics

In [15]:
case_demog = pd.read_csv('E:/2023 EHR data/case_demog04082024.csv')

In [16]:
concept = pd.read_csv('E:/2023 EHR data/concept.csv')
concept = concept[['concept_id', 'concept_name']]
concept_dict = dict(zip(concept['concept_id'], concept['concept_name']))

C:\Users\jp4147\AppData\Local\Temp\ipykernel_49128\3372526552.py:1: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  concept = pd.read_csv('E:/2023 EHR data/concept.csv')


In [17]:
from collections import Counter
case_demog['race'] = case_demog['race_concept_id'].replace({k: concept_dict[k] for k in Counter(case_demog['race_concept_id'])})
case_demog['ethnicity'] = case_demog['ethnicity_concept_id'].replace({k: concept_dict[k] for k in Counter(case_demog['ethnicity_concept_id'])})
case_demog['gender'] = case_demog['gender_concept_id'].replace({k: concept_dict[k] for k in Counter(case_demog['gender_concept_id'])})

In [18]:
case_demog['gender'] = case_demog['gender'].replace(['I prefer not to answer', 'PMI: Skip','None of these', 'No matching concept'], 'Unknown')
case_demog['race'] = case_demog['race'].replace(['None Indicated', 'PMI: Skip','None of these', 'More than one population','I prefer not to answer','No matching concept'], 'Unknown')
case_demog['ethnicity'] = case_demog['ethnicity'].replace(['PMI: Skip', 'PMI: Prefer Not To Answer', 'What Race Ethnicity: Race Ethnicity None Of These','No matching concept'], 'Unknown')

In [19]:
print(Counter(case_demog['gender']))
print(Counter(case_demog['race']))
print(Counter(case_demog['ethnicity']))

Counter({'FEMALE': 6387, 'MALE': 6222, 'Unknown': 3})
Counter({'Unknown': 6150, 'White': 5260, 'Black or African American': 846, 'Asian': 271, 'Other Race': 60, 'American Indian or Alaska Native': 14, 'Native Hawaiian or Other Pacific Islander': 10, 'Asian Indian': 1})
Counter({'Unknown': 7050, 'Not Hispanic or Latino': 4395, 'Hispanic or Latino': 1167})


In [20]:
case_demog['date_of_birth'] = case_demog.apply(lambda row: pd.Timestamp(year=row['year_of_birth'],
                                                                         month=row['month_of_birth'],
                                                                         day=row['day_of_birth']), axis=1)

In [21]:
sex_dict = dict(zip(case_demog['person_id'], case_demog['gender']))
race_dict = dict(zip(case_demog['person_id'], case_demog['race']))
eth_dict = dict(zip(case_demog['person_id'], case_demog['ethnicity']))
bd_dict = dict(zip(case_demog['person_id'], case_demog['date_of_birth']))

In [22]:
age = []
for i, row in tqdm(pre.iterrows(), total = pre.shape[0]):
    age.append((row['condition_start_datetime'] - bd_dict[row['person_id']]) / pd.Timedelta(days=365.25))

100%|██████████| 781734/781734 [01:06<00:00, 11695.88it/s]


In [23]:
pre['age'] = age

In [24]:
pre['age'].isna().any()

False

### remove patients with other cancers before PC diagnosis

In [25]:
other_cancer_pat = pre[['person_id', 'condition_concept_id']]
other_cancer_pat.columns = ['person_id', 'concept_id']
other_cancer_pat = other_cancer_pat.merge(concept, on = 'concept_id', how = 'inner')

In [26]:
other_cancer_patlst = []
for i, row in other_cancer_pat.iterrows():
    n = row['concept_name'].lower()
    if 'malig' in n or 'adenocarcinoma' in n:
        other_cancer_patlst.append(row['person_id'])

In [27]:
other_cancer_patlst = list(set(other_cancer_patlst))

In [28]:
len(other_cancer_patlst)

3474

In [29]:
pre_nocancer = pre[~pre['person_id'].isin(other_cancer_patlst)]

In [30]:
len(pre_nocancer['person_id'].unique())

3516

### patient data into dictionary

In [31]:
grk = pre_nocancer.groupby('person_id')
patlst = list(pre_nocancer['person_id'].unique())

pat_dat = {}
la = []
ctrl = []
for p in tqdm(patlst):
    dat = grk.get_group(p)
    if len(dat)>=5:
        dat = dat.sort_values(by = 'age')
        last_diag = list(dat['condition_start_datetime'])[-1]
        month_before = (pc_diag[p]-last_diag).days/ 30.44
        label = []
        if month_before<3:
            la.append('3m')
            label = [1,1,1,1,1]
        elif month_before<6:
            la.append('6m')
            label = [0,1,1,1,1]
        elif month_before<12:
            la.append('12m')
            label = [0,0,1,1,1]
        elif month_before<36:
            la.append('12m')
            label = [0,0,0,1,1]
        elif month_before<60:
            la.append('60m')
            label = [0,0,0,0,1]
        else:
            ctrl.append(p)
        
        if len(label)>0:
            pat_dat[p] = {}
            pat_dat[p]['concept_dx'] = list(dat['condition_concept_id'])
            pat_dat[p]['timestamps'] = list(dat['condition_start_datetime'])
            pat_dat[p]['age'] = list(dat['age'])
            pat_dat[p]['race'] = race_dict[p]
            pat_dat[p]['ethnicity'] = eth_dict[p]
            pat_dat[p]['gender'] = sex_dict[p]
            pat_dat[p]['age_at_diagnosis'] = pc_diag[p]
            pat_dat[p]['birth_date'] = bd_dict[p]
            pat_dat[p]['label'] = label

  0%|          | 0/3516 [00:00<?, ?it/s]

100%|██████████| 3516/3516 [00:03<00:00, 1063.52it/s]


In [32]:
len(pat_dat)

3387

In [33]:
la_lst = []
for p, info in pat_dat.items():
    la_lst.append(sum(info['label']))

In [34]:
Counter(la_lst)

Counter({5: 2908, 2: 175, 3: 119, 4: 115, 1: 70})

In [35]:
import pickle
with open('datasets/case.pickle', 'wb') as handle:
    pickle.dump(pat_dat, handle)

# Control

In [36]:
# control conditions
# SELECT co.person_id, co.condition_concept_id, co.condition_start_datetime
# FROM clinical_merge_v5_2022q1.condition_occurrence co
# WHERE co.person_id NOT IN (
#     SELECT DISTINCT c.person_id
#     FROM clinical_merge_v5_2022q1.condition_occurrence c
#     WHERE c.condition_concept_id IN (
#         SELECT concept.concept_id
#         FROM clinical_merge_v5_2022q1.concept
#         WHERE concept.concept_name LIKE '%malig%'
#         OR concept.concept_name LIKE '%adenocarcinoma%'
#     )
# )

# control demog
# SELECT p.person_id, p.year_of_birth, p.month_of_birth, p.day_of_birth, p.race_concept_id, p.ethnicity_concept_id, p.gender_concept_id
# FROM clinical_merge_v5_2022q1.person p
# WHERE p.person_id NOT IN (
#     SELECT DISTINCT c.person_id
#     FROM clinical_merge_v5_2022q1.condition_occurrence c
#     WHERE c.condition_concept_id IN (
#         SELECT concept.concept_id
#         FROM clinical_merge_v5_2022q1.concept
#         WHERE concept.concept_name LIKE '%malig%'
#         OR concept.concept_name LIKE '%adenocarcinoma%'
#     )
# )

In [37]:
control = pd.read_csv('E:/2023 EHR data/control_condition04082024.csv')

### Remove concept id 0 - unknown

In [43]:
len(control)

134234023

In [44]:
control = control[control['condition_concept_id']!=0]
len(control)

133019266

### Control demographics

In [38]:
demog = pd.read_csv('E:/2023 EHR data/control_demog04082024.csv')

In [ ]:
concept = pd.read_csv('E:/2023 EHR data/concept.csv')
concept = concept[['concept_id', 'concept_name']]
concept_dict = dict(zip(concept['concept_id'], concept['concept_name']))

C:\Users\jp4147\AppData\Local\Temp\ipykernel_10728\3372526552.py:1: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  concept = pd.read_csv('E:/2023 EHR data/concept.csv')


In [39]:
from collections import Counter
demog['race'] = demog['race_concept_id'].replace({k: concept_dict[k] for k in Counter(case_demog['race_concept_id'])})
demog['ethnicity'] = demog['ethnicity_concept_id'].replace({k: concept_dict[k] for k in Counter(case_demog['ethnicity_concept_id'])})
demog['gender'] = demog['gender_concept_id'].replace({k: concept_dict[k] for k in Counter(case_demog['gender_concept_id'])})

demog['gender'] = demog['gender'].replace(['I prefer not to answer', 'PMI: Skip','None of these', 'No matching concept'], 'Unknown')
demog['race'] = demog['race'].replace(['None Indicated', 'PMI: Skip','None of these', 'More than one population','I prefer not to answer','No matching concept'], 'Unknown')
demog['ethnicity'] = demog['ethnicity'].replace(['PMI: Skip', 'PMI: Prefer Not To Answer', 'What Race Ethnicity: Race Ethnicity None Of These','No matching concept'], 'Unknown')

print(Counter(demog['gender']))
print(Counter(demog['race']))
print(Counter(demog['ethnicity']))

Counter({'FEMALE': 3581329, 'MALE': 2807802, 'Unknown': 17176})
Counter({'Unknown': 4848120, 'White': 988520, 'Black or African American': 372778, 'Other Race': 104061, 'Asian': 78276, 'American Indian or Alaska Native': 7259, 'Native Hawaiian or Other Pacific Islander': 7005, 38003613: 198, 'Asian Indian': 29, 38003584: 18, 38003579: 17, 38003585: 9, 38003581: 9, 44814649: 3, 38003610: 1, 38003595: 1, 38003583: 1, 38003586: 1, 38003591: 1})
Counter({'Unknown': 4928128, 'Not Hispanic or Latino': 953890, 'Hispanic or Latino': 524289})


In [40]:
demog['race'] = demog['race'].replace([38003613,44814649,38003579,38003610,38003584,38003585,38003581,38003595,38003583,38003586,38003591], 'Unknown')
demog['race'] = demog['race'].replace(['Asian Indian'], 'Asian')

In [41]:
Counter(demog['race'])

Counter({'Unknown': 4848379,
         'White': 988520,
         'Black or African American': 372778,
         'Other Race': 104061,
         'Asian': 78305,
         'American Indian or Alaska Native': 7259,
         'Native Hawaiian or Other Pacific Islander': 7005})

In [45]:
print(len(control['person_id'].unique()))
print(len(control['condition_concept_id'].unique()))

4388035
15572


In [46]:
print(len(demog['person_id'].unique()))
print(len(control['person_id'].unique()))

6406307
4388035


In [47]:
demog['date_of_birth'] = demog.apply(lambda row: pd.Timestamp(year=row['year_of_birth'], month=row['month_of_birth'],day=row['day_of_birth']), axis=1)

In [48]:
sex_dict = dict(zip(demog['person_id'], demog['gender']))
race_dict = dict(zip(demog['person_id'], demog['race']))
eth_dict = dict(zip(demog['person_id'], demog['ethnicity']))
bd_dict = dict(zip(demog['person_id'], demog['date_of_birth']))

In [49]:
control['condition_start_datetime'] = pd.to_datetime(control['condition_start_datetime'], errors = 'coerce')

### Cut recent 2 years of data

In [50]:
import pandas as pd
from tqdm import tqdm

# Step 1: Sort the DataFrame by 'person_id' and 'condition_start_datetime'
control = control.sort_values(by=['person_id', 'condition_start_datetime'])

# Calculate the last diagnosis date for each person_id minus two years
last_diag_minus_two_years = control.groupby('person_id')['condition_start_datetime'].transform('max') - pd.Timedelta(days=730)

# Filter the DataFrame to only include dates before two years from the last diagnosis
control['within_two_years'] = control['condition_start_datetime'] < last_diag_minus_two_years

# Now, filter to keep only those within the two-year period
control_filtered = control[control['within_two_years']]

# Step 2: Ensure at least 5 records exist for the condition within the timeframe for each person_id
# This involves a groupby and filter operation
control_filtered = control_filtered.groupby('person_id').filter(lambda x: len(x) >= 5)

In [51]:
control_filtered = control_filtered[~control_filtered.duplicated()]

In [52]:
print(len(control_filtered['person_id'].unique()))

1091311


In [53]:
control_filtered['age'] = (control_filtered['condition_start_datetime'] - control_filtered['person_id'].map(bd_dict)) / pd.Timedelta(days=365.25)

### patient data into dictionary

In [54]:
def aggregate_group_data(name, group):
    return {
        'concept_dx': group['condition_concept_id'].tolist(),
        'timestamps': group['condition_start_datetime'].tolist(),
        'race': race_dict[name], 
        'ethnicity': eth_dict[name],  
        'gender': sex_dict[name],  
        'birth_date': bd_dict[name],  
        'label': [0, 0, 0, 0, 0],
        'age': group['age'].tolist()
    }

In [55]:
groups = list(control_filtered.groupby('person_id'))

In [56]:
pat_dat = {}
for name, group in tqdm(groups, total=len(groups)):
    aggregated_data = aggregate_group_data(name, group)
    pat_dat[name] = aggregated_data

  0%|          | 0/1091311 [00:00<?, ?it/s]

100%|██████████| 1091311/1091311 [05:07<00:00, 3546.21it/s]


In [57]:
len(pat_dat)

1091311

In [58]:
with open('datasets/control.pickle', 'wb') as handle:
    pickle.dump(pat_dat, handle)

# Combine case and control

In [64]:
import pickle
with open('datasets/case.pickle', 'rb') as handle:
    case = pickle.load(handle)

In [65]:
with open('datasets/control.pickle', 'rb') as handle:
    control = pickle.load(handle)

In [66]:
pat_data = {}

pat_data.update(case)
pat_data.update(control)
len(pat_data)

1094698

In [67]:
with open('datasets/pat_data.pickle', 'wb') as handle:
    pickle.dump(pat_data, handle)

# Age exclusion

In [1]:
import pickle
with open('datasets/pat_data.pickle', 'rb') as handle:
    pat_data = pickle.load(handle)

In [ ]:
from tqdm import tqdm
pids_rm = []
for k,v in tqdm(pat_data.items()):
    if any([number < 18 for number in v['age']]):
        pids_rm.append(k)
    elif any([number > 100 for number in v['age']]):
        pids_rm.append(k)

In [ ]:
pids = list(pat_data.keys())

In [ ]:
pids = list(set(pids) - set(pids_rm))
pat_data = {i:pat_data[i] for i in pids}

# Age normalization

In [ ]:
from tqdm import tqdm
# Calculate the minimum and maximum ages
all_ages = []
p = []
for k,v in tqdm(pat_data.items()):
    all_ages.extend(v['age'])
    if 0 in v['age']:
        p.append(k)
min_age = min(all_ages)
max_age = max(all_ages)

# Normalize the ages using the adjusted min-max scaling
for k,v in tqdm(pat_data.items()):
    v['age_norm'] = [
        (0.1 + 0.9 * ((age - min_age) / (max_age - min_age))) for age in v['age']
    ]

In [ ]:
with open('datasets/pat_data.pickle', 'wb') as handle:
    pickle.dump(pat_data, handle)